# NN.TRANSFORMER 와 TORCHTEXT 로 시퀀스-투-시퀀스(SEQUENCE-TO-SEQUENCE) 모델링하기

이 튜토리얼에서는 nn.Transformer 모듈을 이용하는 Seq2Seq 모델을 학습하는 방법을 배워보겠습니다.

PyTorch 1.2 버젼에는 Attention is All You Need 논문에 기반한 표준 트랜스포머(transformer) 모듈을 포함하고 있습니다. 트랜스포머 모델은 다양한 seq2seq 문제들에서 더 병렬화가 가능하면서 RNN과 비교하여 더 나은 성능을 보임이 입증되었다. 

nn.Transformer 모듈은 입력과 출력 사이의 전역적인 의존성을 나타내기 위해 nn.MultiheadAttention에 전적으로 의존한다. 

현재 nn.Transformer 모듈은 모듈화가 잘 되어 있어 단일 컴포넌트로 쉽게 적용 및 구성할 수 있다.

![](/Users/junghwankim/Desktop/pytorch_tutorial/Seq2Seq_tutorial/transformer_architecture.jpeg)

# 모델 정의하기

이 튜토리얼에서 우리는 nn.TransformerEncoder 모델을 Language modeling 과제에 대해서 학습시킬 것이다. 언어 모델링 과제는 주어진 단어가 다음에 이어지는 단어 시퀀스를 따를 likelihood에 대한 확률을 할당하는 것이다. 

먼저, 토큰들의 시퀀스가 임베딩 레이어로 전달되며, 이어서 포지셔널 인코딩 레이어가 각 단어의 순서를 설명한다. 

nn.TransformerEncoder는 여러 개의 nn.TransformerEncoderLayer로 구성되어 있다. nn.TransformerEncoder 내부의 셀프-어텐션 레이어들은 시퀀스 안에서의 이전 포지션에만 집중하도록 허용되기 때문에 입력순서와 함께 정사각 형태의 어텐션 마스크가 필요하다. 언어 모델링 과제를 위해 미래의 포지션에 있는 모든 토큰들은 마스킹 되어 가려져야 한다. 실제 단어를 얻기 위해, nn.TransformerEncoder의 출력은 로그-소프트맥스로 이어지는 최종 선형 레이어로 전달된다.

In [1]:
import math
from typing import Tuple 

import torch
from torch import nn, Tensor
import torch.nn.functional as F
from torch.nn import TransformerEncoder, TransformerEncoderLayer
from torch.utils.data import dataset

In [ ]:
class TransformerModel(nn.Module):

    def __init__(self, ntoken: int, d_model: int, nhead: int, d_hid: int, nlayers: int, dropout: float = 0.5):
        super().__init__()
        self.model_type = 'Transformer'
        self.pos_encoder = PositionalEncoding(d_model, dropout)
        encoder_layers = TransformerEncoderLayer(d_model, nhead, d_hid, dropout)
        self.transformer_encoder = TransformerEncoder(encoder_layers, nlayers)
        self.encoder = nn.Embedding(ntoken, d_model)
        self.d_model = d_model
        self.decoder = nn.Linear(d_model, ntoken)

        self.init_weights()

    def init_weights(self) -> None:
        initrange = 0.1
        self.encoder.weight.data.uniform_(-initrange, initrange)
        self.decoder.bias.data.zero_()
        self.decoder.weight.data.uniform_(-initrange, initrange)
    
    def forward(self, src : Tensor, src_mask: Tensor -> Tensor):
